---
layout: post
title:  Scala 2 Closures
date:   2025-10-24
categories: [Scala]
mermaid: true
typora-root-url: /Users/ojitha/GitHub/ojitha.github.io
typora-copy-images-to: ../assets/images/${filename}
---

<style>
/* Styles for the two-column layout */
.image-text-container {
    display: flex; /* Enables flexbox */
    flex-wrap: wrap; /* Allows columns to stack on small screens */
    gap: 20px; /* Space between the image and text */
    align-items: center; /* Vertically centers content in columns */
    margin-bottom: 20px; /* Space below this section */
}

.image-column {
    flex: 1; /* Allows this column to grow */
    min-width: 250px; /* Minimum width for the image column before stacking */
    max-width: 40%; /* Maximum width for the image column to not take up too much space initially */
    box-sizing: border-box; /* Include padding/border in element's total width/height */
}

.text-column {
    flex: 2; /* Allows this column to grow more (e.g., twice as much as image-column) */
    min-width: 300px; /* Minimum width for the text column before stacking */
    box-sizing: border-box;
}

</style>

<div class="image-text-container">
    <div class="image-column">
        <img src="/assets/images/2025-07-25-Scala-basics/scala_basics.png" alt="Scala basics" width="150" height="150">
    </div>
    <div class="text-column">
<p>Explore fundamental Scala programming concepts including its functional and object-oriented nature, immutable variables , mutable variables, type inference, and basic syntax for defining functions and classes. This guide introduces key Scala features for developers familiar with other programming languages looking to understand its core principles and get started with Scala development. Discover the power and expressiveness of Scala through this concise overview of its building blocks.</p>
    </div>
</div>

<!--more-->

------

* TOC
{:toc}
------

## What is a Closure?

A **closure** is a function value (object) created at runtime from a function literal that captures (or "closes over") **free variables** from its surrounding scope.[^1].

> A closure = A function + The variables it needs from outside its own scope.
{:.info-box}

When you create a function that *references variables from outside its parameter list, Scala automatically captures those variables so the function can access them <span>even after leaving the original scope</span>{:gtxt}*.

| Concept | Definition | Example |
|---------|------------|---------|
| **Function Literal** | Source code representation of a function | `(x: Int) => x + 1` |
| **Function Value** | Runtime object instance of a function | Object created from literal |
| **Free Variable** | Variable used but not defined in function | `more` in `(x: Int) => x + more` |
| **Bound Variable** | Variable defined as function parameter | `x` in `(x: Int) => x + more` |
| **Closed Term** | Function with no free variables | `(x: Int) => x + 1` |
| **Open Term** | Function with free variables | `(x: Int) => x + more` |
| **Closure** | Function value that captures free variables | Runtime object with captured vars |

Closure Creation Flow:

```mermaid
---
config:
  look: neo
  theme: default
---
flowchart TD
    A[Define Variable in Outer Scope] --> B[Create Function Literal with Free Variable]
    B --> C{Compile Time}
    C --> D[Compiler Detects Free Variable 'more']
    C --> E[Identifies Bound Variable 'x']
    D --> F[Runtime: Create Closure]
    E --> F
    F --> G[Closure Captures Reference to 'more']
    G --> H[Function Value Ready to Use]
    H --> I[Can Access 'more' Even After Scope Exit]
```
Variable Types in Functions

```mermaid
---
config:
  look: neo
  theme: default
---
graph LR
    A[Function: x Int => x + more] --> B{Variable Analysis}
    B --> C[x: Bound Variable]
    B --> D[more: Free Variable]
    C --> E[Defined in parameter list]
    D --> F[Defined outside function]
    F --> G[Must be captured in closure]
    E --> H[No capture needed]
```

### Variable Capture Mechanisms

Scala closures capture **variables themselves**, not just their values. This is a critical distinction.[^1]

```mermaid
---
config:
  look: neo
  theme: default
---
sequenceDiagram
    participant Outer Scope
    participant Closure
    participant Heap
    
    Note over Outer Scope: var more = 1
    Outer Scope->>Heap: Store 'more' reference
    Note over Outer Scope: val addMore = (x: Int) => x + more
    Outer Scope->>Closure: Create closure
    Closure->>Heap: Capture reference to 'more'
    Note over Outer Scope: more = 9999
    Outer Scope->>Heap: Update 'more' value
    Note over Closure: addMore(10) called
    Closure->>Heap: Read current 'more' value
    Heap-->>Closure: Returns 9999
    Closure-->>Closure: Compute 10 + 9999 = 10009
```

### Capture Behavior Comparison

| Aspect | Scala Closure Behavior | Alternative (Value Capture) |
|--------|------------------------|----------------------------|
| **What's Captured** | Reference to variable | Copy of value at creation |
| **Sees External Changes** | ✅ Yes | ❌ No |
| **Can Modify Variable** | ✅ Yes (if `var`) | ❌ No |
| **Memory Location** | Heap (if needed) | Stack/Inline |
| **Lifetime** | Can outlive creating function | Limited to closure lifetime |



[^1]: Programming in Scala Fourth Edition, Ch. 8: "Functions and Closures" → "Closures"


{:gtxt: .message color="green"}
{:ytxt: .message color="yellow"}
{:rtxt: .message color="red"}